In [ ]:
from google.colab import drive
import pickle
drive.mount('/content/drive')


Mounted at /content/drive


**Data Extraction**

In [ ]:
import os
import pandas as pd
import re
extracted_emotion = []
extracted_transcription = []
audio_feature = []
emotion = []
def extract_transcription(path):
    with open(path, 'r') as file:
        data = file.read()



    pattern = r'(?P<filename>\S+) \[(?P<time>\d+\.\d+-\d+\.\d+)\]: (?P<text>.+)'


    for match in re.finditer(pattern, data):
        extracted_transcription.append(match.groupdict())

def extract_emotion_from_file(file_path):

    with open(file_path, 'r') as file:
        data = file.read()


    pattern = r'\[(?P<time>\d+\.\d+ - \d+\.\d+)\]\s+(?P<filename>\S+)\s+(?P<emotion>\S+)\s+\[(?P<scores>[\d\.,\s]+)\]'


    for match in re.finditer(pattern, data):
        extracted_emotion.append(match.groupdict())


def crawl_directory(path, target_directories):
    for root, dirs, files in os.walk(path):
       # print(f'Current directory: {root}')
         print("ROOT:",root)
         #Filtering directories
         dirs[:] = [d for d in dirs if d in target_directories]

#         for dir_name in dirs:
#             dir_path = os.path.join(root, dir_name)
#             #print(f'Directory: {dir_path}')
         print("DIR:",dirs)

         for file_name in files:
            file_path = os.path.join(root, file_name)
            if root == "/content/drive/MyDrive/Session1/dialog/EmoEvaluation":


                extract_emotion_from_file(file_path)
            elif root == "/content/drive/MyDrive/Session1/dialog/transcriptions":
                extract_transcription(file_path)



# Example usage: Crawl only through directories named 'target_dir'
path = "/content/drive/MyDrive/Session1"
target_dirs = ['Session1','dialog','EmoEvaluation','transcriptions']
crawl_directory(path, target_dirs)


ROOT: /content/drive/MyDrive/Session1
DIR: ['dialog']
ROOT: /content/drive/MyDrive/Session1/dialog
DIR: ['transcriptions', 'EmoEvaluation']
ROOT: /content/drive/MyDrive/Session1/dialog/transcriptions
DIR: []
ROOT: /content/drive/MyDrive/Session1/dialog/EmoEvaluation
DIR: []


In [ ]:
extracted_emotion[:2]

[{'time': '6.2100 - 9.3200',
  'filename': 'Ses01F_script01_1_F000',
  'emotion': 'fru',
  'scores': '2.0000, 2.3333, 2.3333'},
 {'time': '9.3500 - 12.8955',
  'filename': 'Ses01F_script01_1_F001',
  'emotion': 'xxx',
  'scores': '2.0000, 2.3333, 1.6667'}]

In [ ]:
data_emotion = pd.DataFrame(extracted_emotion)
data_transcriptions = pd.DataFrame(extracted_transcription)



In [ ]:
data_emotion['text']=None

In [ ]:
for i in range(len(data_emotion)):
    filename = data_emotion.iloc[i].filename
    x = data_transcriptions[data_transcriptions['filename'] == filename].text
    data_emotion.iloc[i,data_emotion.columns.get_loc('text')] = x.iloc[0]

**DataFrame for each session**

In [ ]:
data_emotion

,time,filename,emotion,scores,text
0,6.2100 - 9.3200,Ses01F_script01_1_F000,fru,"2.0000, 2.3333, 2.3333",What's he going to say?
1,9.3500 - 12.8955,Ses01F_script01_1_F001,xxx,"2.0000, 2.3333, 1.6667",Maybe we should tell him before he sees it.
2,14.3063 - 19.5526,Ses01F_script01_1_F002,sur,"2.3333, 2.3333, 2.6667",How could he see it? I was the first one up; ...
3,22.3200 - 24.6667,Ses01F_script01_1_F003,xxx,"3.0000, 3.0000, 2.6667",When?
4,35.3799 - 39.0900,Ses01F_script01_1_F004,xxx,"2.0000, 2.0000, 2.0000",What was he doing out here at four o clock in ...
...,...,...,...,...,...
1814,497.8775 - 500.0400,Ses01M_script02_2_M050,neu,"3.0000, 2.5000, 1.5000",It's not champagne.
1815,509.1461 - 513.8400,Ses01M_script02_2_M051,neu,"4.5000, 4.5000, 1.0000","Are you cold, huh? Do you want to go home?"
1816,525.2750 - 528.0391,Ses01M_script02_2_M052,hap,"5.0000, 4.0000, 1.5000",Shh...
1817,528.9636 - 535.2345,Ses01M_script02_2_M053,hap,"5.0000, 4.5000, 1.5000","If we are really quiet, the fish might come."


In [ ]:
text_data = data_emotion.copy()

In [ ]:
text_data['emotion'] = text_data['emotion'].replace('xxx','neu')

In [ ]:
text_data

**Saving DataFrame as pickle file**

In [ ]:
path = '/content/drive/MyDrive/dataframe_ses1.pickle'


# Open the file in binary write mode
with open(path, 'wb') as f:
    # Dump the data into the file
    pickle.dump(text_data, f)